In [1]:
from datasets import load_dataset, Dataset, Features, Array2D, concatenate_datasets
import glob
import os
import torch
import torchaudio

In [2]:
sources = glob.glob("MYRiAD_V2/audio/SRC/*.wav")
cocktail_party = [f"CP{i}.wav" for i in range(6)];
source_list = [suffix[-6:] for suffix in sources] + cocktail_party 
inner = [f"CMA10_{a}_" for a in [0,90,180,-90]]
outer = [f"CMA20_{a}_" for a in [0,45,90,135,180,-135,-90,-45]]
channel_list = inner + outer
ref_channel = channel_list[0]

In [3]:
dataset_list = []

for source in source_list:
    query = ref_channel+source
    prefix_list = glob.glob("MYRiAD_V2/audio/*/*/*/"+query)
    prefix_list = [match[:-len(query)] for match in prefix_list]
    for prefix in prefix_list:
        x = [];
        try:
            for channel in channel_list:
                filename = prefix + channel + source
                x.append(torchaudio.load(filename)[0][0])
            x = torch.stack(x)
            C, L = x.shape
            new_L = (L // 65536) * 65536
            start_L = (L - new_L) // 2
            y = x[:, start_L:start_L + new_L]

            tiles = []
            for ℓ in range(0, new_L, 65536):
                tile = y[:, ℓ:ℓ+65536]
                tiles.append(tile)
            features = Features({'acoustic': Array2D(shape=(12, 65536), dtype='float32')})
            dataset = Dataset.from_dict({"acoustic": tiles}, features=features)
            dataset_list.append(dataset)
        except:
            pass
dataset = concatenate_datasets(dataset_list)
dataset = dataset.with_format("torch")
dataset.push_to_hub("danjacobellis/MYRiAD_V2")

Pushing dataset shards to the dataset hub:   0%|          | 0/34 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/tuple'
  in "<unicode string>", line 6, column 16:
            shape: !!python/tuple
                   ^